In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline


from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV


from sklearn.metrics import accuracy_score

In [15]:
data_df = pd.read_csv("datasets\housing-classification-iter-0-2\housing-classification-iter-0-2.csv")
data_df.describe()

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,70.049958,1057.429452,2.866438,0.613014,2.758904,1.767123,94.244521,15.060959,0.148630
std,9981.264932,24.284752,438.705324,0.815778,0.644666,40.177307,0.747315,125.338794,55.757415,0.355845
min,1300.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7553.500000,59.000000,795.750000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,9478.500000,69.000000,991.500000,3.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000
75%,11601.500000,80.000000,1298.250000,3.000000,1.000000,0.000000,2.000000,168.000000,0.000000,0.000000
max,215245.000000,313.000000,6110.000000,8.000000,3.000000,738.000000,4.000000,857.000000,480.000000,1.000000


In [16]:
data_df = data_df.drop(columns=["PoolArea","ScreenPorch"])

In [17]:
y = data_df.pop("Expensive")
X = data_df

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=123)

#### Pipeline

In [19]:
my_imputer = SimpleImputer()
my_scaler = StandardScaler()
my_model = KNeighborsClassifier()

In [24]:
pipe = Pipeline(steps=[('impute', my_imputer),('scale', my_scaler), ('classify', my_model)])

#### parameters Range

In [83]:
param_range = {
    "impute__strategy":["median","mean"],
    "scale__with_mean":[True],
    "scale__with_std":[True],
    "classify__n_neighbors" : range(10,15),
    "classify__weights" : ['uniform'],
    "classify__algorithm" : ['auto']
}

#### Random Search

In [73]:
find = RandomizedSearchCV(
    pipe,
    param_range,
    n_iter= 100,
    scoring='accuracy',
    cv = 5,
    verbose= 1,
    random_state=123
)

In [74]:
find.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Python39\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 10 is smaller than n_iter=100. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                             ('scale', StandardScaler()),
                                             ('classify',
                                              KNeighborsClassifier())]),
                   n_iter=100,
                   param_distributions={'classify__algorithm': ['auto'],
                                        'classify__n_neighbors': range(10, 15),
                                        'classify__weights': ['distance'],
                                        'impute__strategy': ['median', 'mean'],
                                        'scale__with_mean': [True],
                                        'scale__with_std': [True]},
                   random_state=123, scoring='accuracy', verbose=1)

In [75]:
find.best_params_

{'scale__with_std': True,
 'scale__with_mean': True,
 'impute__strategy': 'mean',
 'classify__weights': 'distance',
 'classify__n_neighbors': 14,
 'classify__algorithm': 'auto'}

#### Grid Search

In [84]:
search = GridSearchCV(
    pipe,
    param_range,
    scoring='accuracy',
    cv = 5,
    verbose= 1
)

In [85]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                       ('scale', StandardScaler()),
                                       ('classify', KNeighborsClassifier())]),
             param_grid={'classify__algorithm': ['auto'],
                         'classify__n_neighbors': range(10, 15),
                         'classify__weights': ['uniform'],
                         'impute__strategy': ['median', 'mean'],
                         'scale__with_mean': [True],
                         'scale__with_std': [True]},
             scoring='accuracy', verbose=1)

In [86]:
search.best_params_

{'classify__algorithm': 'auto',
 'classify__n_neighbors': 13,
 'classify__weights': 'uniform',
 'impute__strategy': 'median',
 'scale__with_mean': True,
 'scale__with_std': True}

In [87]:
search.best_score_

0.9195260628737024

In [88]:
accuracy_score(y_train,search.predict(X_train))

0.9238013698630136

In [89]:
accuracy_score(y_test,search.predict(X_test))

0.9315068493150684